In [9]:
# Configuration
CLUSTER_ID = 61
output_file = f'cluster_{CLUSTER_ID}.png'

# Generate and display
create_cluster_png(CLUSTER_ID, output_file)
display(Image(filename=output_file))

NameError: name 'create_cluster_png' is not defined

In [11]:
def create_cluster_png(cluster_id, output_path):
    """
    Generates a static PNG map for a specific cluster using Matplotlib and Contextily.
    """
    # Find the cluster
    cluster = next((c for c in data['clusters'] if c['cluster_id'] == cluster_id), None)
    if not cluster:
        print(f"Cluster {cluster_id} not found.")
        return None

    # Center coordinates
    center_lat = cluster['statistics']['geographic_center']['latitude']
    center_lon = cluster['statistics']['geographic_center']['longitude']
    
    # Create Center GeoDataFrame
    center_geom = [Point(center_lon, center_lat)]
    center_gdf = gpd.GeoDataFrame(geometry=center_geom, crs="EPSG:4326")
    
    # Project to UTM for accurate buffering (meters)
    # We use estimate_utm_crs to find the best local projection
    utm_crs = center_gdf.estimate_utm_crs()
    center_utm = center_gdf.to_crs(utm_crs)
    
    # Setup Plot
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # --- Draw Cones (Predictions) ---
    if 'predictions' in cluster:
        p = cluster['predictions']
        
        # Helper to plot buffer
        def plot_buffer(radius_ft, color, alpha, linestyle='-'):
            radius_m = radius_ft * 0.3048
            buffer_utm = center_utm.buffer(radius_m)
            # Reproject to Web Mercator (EPSG:3857) for Contextily
            buffer_web = buffer_utm.to_crs(epsg=3857)
            buffer_web.plot(ax=ax, color=color, alpha=alpha, edgecolor=color, linestyle=linestyle, linewidth=2)

        # 90% (Severe) - Red
        plot_buffer(p['radius_90_severe'], '#c0392b', 0.1, linestyle='--')

        # 50% (Expected) - Orange
        plot_buffer(p['radius_50_expected'], '#f39c12', 0.2)

        # 10% (Conservative) - Green
        plot_buffer(p['radius_10_conservative'], '#27ae60', 0.3)

    # --- Draw Cases ---
    if 'cases' in cluster:
        cases_data = []
        for case in cluster['cases']:
            cases_data.append(Point(case['longitude'], case['latitude']))
        
        if cases_data:
            cases_gdf = gpd.GeoDataFrame(geometry=cases_data, crs="EPSG:4326")
            cases_web = cases_gdf.to_crs(epsg=3857)
            cases_web.plot(ax=ax, color='white', edgecolor='#2c3e50', markersize=50, linewidth=1, zorder=10)

    # --- Draw Center Marker ---
    center_web = center_gdf.to_crs(epsg=3857)
    center_web.plot(ax=ax, color='#2c3e50', edgecolor='white', markersize=100, linewidth=1, zorder=11)

    # --- Add Basemap ---
    try:
        # Use CartoDB Positron to match dashboard style
        ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
    except Exception as e:
        print(f"Could not fetch basemap: {e}")

    # Remove axes
    ax.set_axis_off()
    
    # Save
    plt.savefig(output_path, dpi=150, bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    print(f"Saved PNG to {output_path}")
    return output_path

In [10]:
import json
import os
import pandas as pd
import matplotlib.pyplot as plt
import contextily as ctx
import geopandas as gpd
from shapely.geometry import Point
from IPython.display import display, Image

# Path to the dashboard data
DATA_PATH = '../visuals/dashboard_data.js'

def load_dashboard_data():
    """Loads the dashboard data from the JS file."""
    with open(DATA_PATH, 'r') as f:
        content = f.read()
        # Strip the variable declaration 'var dashboardData = ' and the trailing semicolon if present
        json_str = content.replace('var dashboardData = ', '').strip()
        if json_str.endswith(';'):
            json_str = json_str[:-1]
        return json.loads(json_str)

data = load_dashboard_data()
print(f"Loaded {len(data['clusters'])} clusters.")

Loaded 79 clusters.


# Generate Cluster Images
This notebook generates static images (maps) of oak wilt clusters and their risk cones, replicating the visual style of the web dashboard.

In [15]:
# Generate and save map for the selected cluster
output_dir = '../visuals/cluster_images'
os.makedirs(output_dir, exist_ok=True)

filename = f"{output_dir}/cluster_{CLUSTER_ID}.png"
create_cluster_png(CLUSTER_ID, filename)

Saved PNG to ../visuals/cluster_images/cluster_61.png


'../visuals/cluster_images/cluster_61.png'